In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from yt.utilities.exceptions import YTInvalidFieldType
import ipywidgets as widgets
import traitlets
from IPython.display import HTML
import yt
plt.ioff()

In [ ]:
from yt.funcs import mylog
mylog.setLevel(50) # This sets the log level to "critical"  
# refer to https://yt-project.org/doc/faq/index.html#how-can-i-change-yt-s-log-level

In [2]:
# data source
f = open("data_path.txt", "r")
DATA_PATH =f.read()

str

In [ ]:
EXPLANATION = """\
<div class="app-sidebar">

<p>This example demonstrates using different width_value, field_name and certain type of plot to generate different images.
</div>
"""

In [ ]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [ ]:
class App:

    def __init__(self, ds):
        
        self._ds=ds
        self._indicators=self._create_valid_indicator(ds)
        
        self._radiobutton=self._create_radiobutton()
        self._dropdown, dropdown_box = self._create_indicator_dropdown(self._indicators, 0)
        self._width_slider, width_slider_box = self._create_width_slider(0.01, 1.0,0.01)
        
        self._plot_container_x = widgets.Output()
        self._plot_container_y = widgets.Output()
        self._plot_container_z = widgets.Output()
        
        _app_container = widgets.TwoByTwoLayout(
            top_left=self._plot_container_x,
            top_right=self._plot_container_y,
            bottom_left=self._plot_container_z)
        
        
        
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>This is a demo dashboard for yt Sliceplot & Projectionplot</h1>'
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            widgets.HBox([self._radiobutton, dropdown_box,width_slider_box]),
            widgets.HBox([
                _app_container, 
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 1em'))
            ])
        ], 
            layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        
        self._update_app()
        
    @classmethod
    def load_ds(cls,data_path):
        ds=yt.load(data_path)
        return cls(ds)
    
    def _create_radiobutton(self):
        radio_button=widgets.RadioButtons(options=['SlicePlot', 'ProjectionPlot'],value='SlicePlot', layout={'width': 'max-content'}, disabled=False)
        radio_button.observe(self._on_change, names=['value'])
        return radio_button
    
    def _create_valid_indicator(self,ds):
        field_lst=list(pd.DataFrame(ds.field_list)[1])
        filtered_lst=[]
        for field_name in field_lst:
            try:
                yt.SlicePlot(ds, "x", field_name)
            except (YTInvalidFieldType,TypeError):
                pass
            else:
                filtered_lst.append(field_name)
        return(filtered_lst)
    
    
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown_label = widgets.Label('field name: ')
        dropdown = widgets.Dropdown(options=indicators, value="Density", layout={'width': 'max-content', "margin":'0 2em 0 auto'})
        dropdown_box = widgets.HBox([dropdown_label, dropdown],layout=widgets.Layout(margin='0 0 2em 1em'))
        dropdown.observe(self._on_change, names=['value'])
        return dropdown,dropdown_box
    
    
    def _create_width_slider(self, min_width, max_width, step_value):
        width_slider_label = widgets.Label('width value: ')
        width_slider = widgets.FloatSlider(min=min_width, max=max_width,value=0.01,layout=widgets.Layout(width='250px'))
        width_slider.observe(self._on_change, names=['value'])
        width_slider_box = widgets.HBox([width_slider_label, width_slider])
        return width_slider, width_slider_box
    
    
    
    def _create_plot(self,plot_type,axis,field_name, width_value):
        ds=self._ds
        if plot_type=="SlicePlot":
            slc_x = yt.SlicePlot(ds, axis, field_name)
        else:
            slc_x = yt.ProjectionPlot(ds, axis, field_name)
        slc_x.set_width(width_value)
        slc_x.save("tmp.png")
        im_x= open("tmp.png", "rb").read()
        image_widget_x=widgets.Image(value=im_x, format='png',width=600,height=800)
        return(image_widget_x)
    
    
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        width_value=self._width_slider.value
        field_name=self._dropdown.value
        plot_type=self._radiobutton.value

        self._plot_container_x.clear_output(wait=True)
        with self._plot_container_x:
            display(self._create_plot(plot_type,"x", field_name, width_value))
            
        self._plot_container_y.clear_output(wait=True)
        with self._plot_container_y:
            display(self._create_plot(plot_type,"y",field_name, width_value))
            
        self._plot_container_z.clear_output(wait=True)
        with self._plot_container_z:
            display(self._create_plot(plot_type,"z", field_name, width_value))

In [ ]:
app = App.load_ds(DATA_PATH)
app.container